# Aprendizaje No Supervisado con H2O

### Docente: M.Sc. Richard Fernández 

In [1]:
! pip install h2o

In [ ]:
import h2o

In [51]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,1 hour 2 mins
H2O_cluster_timezone:,America/Lima
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,6 months and 26 days
H2O_cluster_name:,H2O_from_python_HP_m9v49a
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.423 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [91]:
# Carga de datos en el cluster H2O
data_h2o = h2o.import_file(
                path   = "D:/Bases/Otros/inv_mercados.csv",
                header = 1,
                sep    = ",",
                destination_frame = "datos_h2o"
            )

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [92]:
data_h2o.head()

ID,V1,V2,V3,V4,V5,V6
1,6,4,7,3,2,3
2,2,3,1,4,5,4
3,7,2,6,4,1,3
4,4,6,4,5,3,6
5,1,3,2,2,6,4
6,6,4,6,3,3,4
7,5,3,6,3,3,4
8,7,3,7,4,1,4
9,2,4,3,3,6,3
10,3,5,3,6,4,6


In [95]:
predictors = ["V1","V2","V3","V4","V5","V6"]
predictors

['V1', 'V2', 'V3', 'V4', 'V5', 'V6']

In [96]:
train, test = data_h2o.split_frame(ratios=[.9], seed=1234)

In [97]:
train.head()

ID,V1,V2,V3,V4,V5,V6
1,6,4,7,3,2,3
2,2,3,1,4,5,4
3,7,2,6,4,1,3
5,1,3,2,2,6,4
6,6,4,6,3,3,4
7,5,3,6,3,3,4
8,7,3,7,4,1,4
9,2,4,3,3,6,3
11,1,3,2,3,5,3
12,5,4,5,4,2,4


In [98]:
test.head()

ID,V1,V2,V3,V4,V5,V6
4,4,6,4,5,3,6
10,3,5,3,6,4,6
19,4,6,3,7,2,7
20,2,3,2,4,7,2


In [99]:
from h2o.estimators import H2OKMeansEstimator

In [100]:
kmeans = H2OKMeansEstimator(k=3,
                            standardize=True,
                            seed=1234)

In [101]:
kmeans.train(x=predictors,
            training_frame=train,
            validation_frame=test)

kmeans Model Build progress: |███████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OKMeansEstimator : K-means
Model Key: KMeans_model_python_1700785174225_7


Model Summary: 
    number_of_rows    number_of_clusters    number_of_categorical_columns    number_of_iterations    within_cluster_sum_of_squares    total_sum_of_squares    between_cluster_sum_of_squares
--  ----------------  --------------------  -------------------------------  ----------------------  -------------------------------  ----------------------  --------------------------------
    16                3                     0                                2                       27.573                           90                      62.427

ModelMetricsClustering: kmeans
** Reported on train data. **

Total Within Cluster Sum of Square Error: 27.57303429075581
Total Sum of Square Error to Grand Mean: 90.0000011282973
Between Cluster Sum of Square Error: 62.426966837541485

Centroid Statistics: 
    centroid    size    within_cluster_sum_of_squares
--  ----------  ------  -------------------------------
    1           8       12.6579
    2           3       8.54205
    3           5       6.37306

ModelMetricsClustering: kmeans
** Reported on validation data. **

Total Within Cluster Sum of Square Error: 7.909061144875498
Total Sum of Square Error to Grand Mean: 21.050213270508504
Between Cluster Sum of Square Error: 13.141152125633006

Centroid Statistics: 
    centroid    size    within_cluster_sum_of_squares
--  ----------  ------  -------------------------------
    1           0       0
    2           3       4.94114
    3           1       2.96792

Scoring History: 
    timestamp            duration    iterations    number_of_reassigned_observations    within_cluster_sum_of_squares
--  -------------------  ----------  ------------  -----------------------------------  -------------------------------
    2023-11-23 20:41:25  0.021 sec   0             nan                                  nan
    2023-11-23 20:41:25  0.026 sec   1             16                                   61.4315
    2023-11-23 20:41:25  0.026 sec   2             0                                    27.573

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [102]:
prediction_train = kmeans.predict(train)
prediction_train

kmeans prediction progress: |████████████████████████████████████████████████████| (done) 100%


predict
0
2
0
2
0
0
0
2
2
0


In [103]:
prediction_train['predict'].unique()

C1
0
2
1


In [104]:
prediction_test = kmeans.predict(test)
prediction_test

kmeans prediction progress: |████████████████████████████████████████████████████| (done) 100%


predict
1
1
1
2


In [105]:
prediction_test['predict'].unique()

C1
2
1


In [121]:
#performance = kmeans.model_performance()
#performance

In [107]:
train["segmento"] = prediction_train["predict"].asfactor()
train.head()

ID,V1,V2,V3,V4,V5,V6,segmento
1,6,4,7,3,2,3,0
2,2,3,1,4,5,4,2
3,7,2,6,4,1,3,0
5,1,3,2,2,6,4,2
6,6,4,6,3,3,4,0
7,5,3,6,3,3,4,0
8,7,3,7,4,1,4,0
9,2,4,3,3,6,3,2
11,1,3,2,3,5,3,2
12,5,4,5,4,2,4,0


In [119]:
train.group_by(['segmento']).count(na ="all").get_frame()

segmento,nrow
0,8
1,3
2,5


In [120]:
train.group_by(['segmento']).mean(['V1','V2','V3','V4','V5','V6'], na="all").get_frame()

segmento,mean_V1,mean_V2,mean_V3,mean_V4,mean_V5,mean_V6
0,5.75,3.625,6,3.125,1.875,3.875
1,3.33333,6,3.33333,6,4,5.66667
2,1.6,3,1.8,3.4,5.2,3.6
